In [1]:
from dotenv import load_dotenv
from openai import AsyncOpenAI
from agents import Agent, Runner, trace, function_tool, OpenAIChatCompletionsModel, input_guardrail, GuardrailFunctionOutput
from typing import Dict
import sendgrid
import os
from sendgrid.helpers.mail import Mail, Email, To, Content
from pydantic import BaseModel

In [2]:
load_dotenv(override=True)

True

## email generator
generate an email that conforms with the structured output you defined in the pydantic object.

In [16]:
from pydantic import BaseModel, Field

class EmailFormatOutput(BaseModel):
    email_address: str = Field(description="the email address to send the email", example="user@example.com")
    subject: str = Field(description="subject of email")
    body_of_emai: str = Field(description="main body of the email")


user_name = 'ali'
email_writer_system_instruction = f"""
your name id {user_name}, and you are a data scientist working at insurance company.
you want to write an email to report a error your received.
You write concise, to the point cold emails that includes technical keywords.
your email include three main sections: email address, subject of the email, and body of email.
"""

email_writer_agent = Agent(
    name="email writer agent",
    instructions=email_writer_system_instruction,
    output_type=EmailFormatOutput,
    model="gpt-4o-mini"
)

user_message = """
you got an error for "DataError: string data, right truncated".
you want to write an email to infra@company.com to report this error
"""

with trace('structured email writer'):
    email = await Runner.run(email_writer_agent, user_message)

print(f'email address: {email.final_output.email_address}')
print(f'subject: {email.final_output.subject}')
print(f'body of email: {email.final_output.body_of_emai}')

email address: infra@company.com
subject: DataError Report: String Data Truncated
body of email: Dear Team,

I am encountering a DataError: "string data, right truncated" during data processing tasks. This issue seems to stem from a character limit being exceeded in the database. 

Could you please investigate this discrepancy? 

Best regards,
Ali 
Data Scientist


## carefull email generator
it use a guardrail for email generatuion.<br>
your guardrial trips if no error report in the email.

### Q. how to create a gaurdrail?
- create a pydantic object that hints what yo are looking for in the guardrail
- create an guardrail agent
- you build a async function that is decorated with `@input_guardrail`
    - inside the  function, you call a user defined gaurdrail agent
    - you set the condition for tripwire_triggered
    - you return  GuardrailFunctionOutput and `tripwire_triggered`
- you pass the guardrail function as a `input_guardrails` to your main calling agent.

In [ ]:
# example that passes the guardrail
from pydantic import BaseModel, Field

# 1. create structured pydantic to hint what yo are looking for
class ErrorCheckOutput(BaseModel):
    is_error_not_exist_in_email: bool = Field(description="indicate whether the email does not contain a technical error")
    error: str = Field(description="the error that is reported in the email")

# 2. create guardrail agent
error_check_system_prompt = "check if there is an error reported in the email"
error_check_guardrail_agent = Agent(
    name="error check agent",
    instructions=error_check_system_prompt,
    output_type=ErrorCheckOutput,
    model='gpt-4o-mini'
)

# 3. create a wrapped  async function for guardrail
@input_guardrail
async def guardrail_against_error(ctx, agent, message):
    result = await Runner.run(error_check_guardrail_agent, message, context=ctx.context)

    is_error_not_exists_in_email = result.final_output.is_error_not_exist_in_email
    return GuardrailFunctionOutput(
        output_info={"found_error": result.final_output.error},
        tripwire_triggered=is_error_not_exists_in_email
    )


# 4. ADD YOUR GUARDRAIL FUNCTION TO YOUR MAIN CALLING AGENT
carefull_email_writer_agent = Agent(
    name="email writer agent",
    instructions=email_writer_system_instruction,
    output_type=EmailFormatOutput,
    model="gpt-4o-mini",
    input_guardrails=[guardrail_against_error]
)

user_message = """
you got an error for "DataError: string data, right truncated".
you want to write an email to infra@company.com to report this error
"""

with trace('structured email writer'):
    email = await Runner.run(carefull_email_writer_agent, user_message)

print(f'email address: {email.final_output.email_address}')
print(f'subject: {email.final_output.subject}')
print(f'body of email: {email.final_output.body_of_emai}')

email address: infra@company.com
subject: Report of DataError: String Data Truncated
body of email: Dear Team,

I am writing to report a recurring issue encountered during data processing. The error message is as follows:

"DataError: string data, right truncated".

This seems to be occurring due to potential data size limitations in the database schema, which may require adjustments. Please investigate this matter to ensure data integrity and processing continuity.

Thank you for your attention to this issue.

Best regards,
Ali 
Data Scientist


In [20]:
# example that fails the guardrail

user_message = """
write an email that say thank you for sending you a new laptop
"""

with trace('structured email writer'):
    email = await Runner.run(carefull_email_writer_agent, user_message)

print(f'email address: {email.final_output.email_address}')
print(f'subject: {email.final_output.subject}')
print(f'body of email: {email.final_output.body_of_emai}')

InputGuardrailTripwireTriggered: Guardrail InputGuardrail triggered tripwire

In [15]:
a = True
b = not a
b

False